You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!uv sync
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import LLM, Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

MODEL="openrouter/x-ai/grok-4-fast:free"
# MODEL = "gemini/gemini-2.5-flash" 

llm = LLM(
    model=MODEL,
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY")
    # model=MODEL,
    # api_key=os.environ.get("GEMINI_API_KEY")
)


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Top 10 biggest companies in the world in 2023"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e198e39f-4705-439a-815a-5c112af87032                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Top 10 biggest companies in the     │
│  world in 2023.                                                                                                 │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Comprehensive Content Plan: Top 10 Biggest Companies in the World in 2023                                    │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│  This content plan outlines the structure and strategy for a blog article titled "Top 10 Biggest Companies in   │
│  the World in 2023." The article aims to educate readers on the largest companies by market capitalization,     │
│  highlighting their roles in the global economy, recent trends, and investment implications. The plan           │
│  prioritizes factual accuracy using data from reliable sources up to the end of 2023, focuses on engaging       │
│  storytelling to inform decisions, and incorporates SEO best practices for visibility. The final article will   │
│  be approximately 1,500–2,000 words, optimized for readability with subheadings, bullet points, infographics    │
│  suggestions, and visuals like charts of market caps.                                                           │
│                                                                                                                 │
│  Key goals:                                                                                                     │
│  - Engage readers with timely insights into market dominance.                                                   │
│  - Provide value through data-driven analysis to help with informed investing or business strategy.             │
│  - Drive traffic via SEO and encourage audience interaction (e.g., comments, shares).                           │
│                                                                                                                 │
│  ## 1. Prioritized Latest Trends, Key Players, and Noteworthy News                                              │
│  ### Latest Trends                                                                                              │
│  - **Tech and AI Dominance**: Technology companies, particularly those in AI, cloud computing, and              │
│  semiconductors, led the rankings. NVIDIA's market cap surged 239% in 2023 due to AI demand, exemplifying the   │
│  "AI boom." Overall, the top 10's combined market cap exceeded $12 trillion by year-end, with tech firms        │
│  accounting for over 70%.                                                                                       │
│  - **Pharma and Biotech Surge**: Eli Lilly entered the top 10 (peaking at #6) thanks to blockbuster             │
│  weight-loss drugs like Mounjaro, reflecting health tech trends amid post-pandemic wellness focus.              │
│  - **Energy and Diversification Shifts**: Traditional giants like Saudi Aramco remained stable but were         │
│  overshadowed by tech; however, energy transition news (e.g., Berkshire Hathaway's renewable investments)       │
│  highlighted sustainability trends.                                                                             │
│  - **Market Volatility**: Geopolitical tensions (e.g., US-China trade) and inflation impacted rankings, with    │
│  Tesla dropping out of the top 5 due to EV competition, while Broadcom rose on semiconductor demand.            │
│  - **ESG and Innovation Focus**: Companies emphasized e

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f1eec85c-cccc-4dbb-8821-93d74dbc0c14                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Top 10 biggest companies in the world in      │
│  2023.                                                                                                          │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 19416b7a-6927-4f0f-b80f-003dc97dba61                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Top 10 Biggest Companies in the World in 2023 by Market Cap                                                  │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  In 2023, the global economy witnessed unprecedented growth in market capitalization, with the top 10 biggest   │
│  companies in the world reaching a combined value exceeding $12 trillion—more than the GDP of entire nations    │
│  like Japan or Germany. This staggering figure underscores the dominance of innovation-driven industries,       │
│  particularly technology and pharmaceuticals, in shaping our modern world. If you're searching for the largest  │
│  companies by market cap in 2023, you've come to the right place. Market capitalization, or market cap, is      │
│  calculated by multiplying a company's share price by its outstanding shares, offering a snapshot of its        │
│  overall size and investor confidence far beyond mere revenue figures.                                          │
│                                                                                                                 │
│  This list, based on closing prices as of December 31, 2023, highlights not just the sheer scale of these       │
│  giants but also the trends propelling them forward. From the AI boom catapulting NVIDIA to new heights to Eli  │
│  Lilly's breakthrough in weight-loss drugs, 2023 was a year of transformation. As an investor or business       │
│  enthusiast, understanding these top global companies to invest in can inform your strategy, whether you're     │
│  navigating stock picks or analyzing industry shifts.                                                           │
│                                                                                                                 │
│  Drawing from reliable sources like Yahoo Finance and CompaniesMarketCap.com, this overview demystifies the     │
│  rankings while providing actionable insights. Whether you're a beginner decoding market cap explained or a     │
│  seasoned pro eyeing S&P 500 leaders, let's uncover how these behemoths influenced everything from your         │
│  smartphone to global supply chains. These companies represent balanced opportunities across sectors, blending  │
│  high-growth tech with stable pharma and diversified investments.                                               │
│                                                                                                                 │
│  ## The Power Players: Ranking the Top 10 Biggest Companies in 2023                                             │
│                                                                                                                 │
│  Diving into the top 10 biggest companies in the world in 2023 reveals a landscape tilted heavily toward tech   │
│  innovation, with a few surprises from pharma and diversified conglomerates. These rankings by market cap       │
│  showcase companies that didn't just survive economic volatility—they thrived, fueled by AI hype, cloud         │
│  computing, and health breakthroughs. To visualize this

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b5d3c628-41fc-42ad-9cd0-01d8d12ae05c                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e198e39f-4705-439a-815a-5c112af87032                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Top 10 Biggest Companies in the World in 2023 by Market Cap                                    │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  In 2023, the global economy witnessed unprecedented growth in market capitalization, with the top 10 biggest   │
│  companies in the world reaching a combined value exceeding $12 trillion—more than the GDP of entire nations    │
│  like Japan or Germany. This staggering figure underscores the dominance of innovation-driven industries,       │
│  particularly technology and pharmaceuticals, in shaping our modern world. If you're searching for the largest  │
│  companies by market cap in 2023, you've come to the right place. Market capitalization, or market cap, is      │
│  calculated by multiplying a company's share price by its outstanding shares, offering a snapshot of its        │
│  overall size and investor confidence far beyond mere revenue figures.                                          │
│                                                                                                                 │
│  This list, based on closing prices as of December 31, 2023, highlights not just the sheer scale of these       │
│  giants but also the trends propelling them forward. From the AI boom catapulting NVIDIA to new heights to Eli  │
│  Lilly's breakthrough in weight-loss drugs, 2023 was a year of transformation. As an investor or business       │
│  enthusiast, understanding these top global companies to invest in can inform your strategy, whether you're     │
│  navigating stock picks or analyzing industry shifts.                                                           │
│                                                                                                                 │
│  Drawing from reliable sources like Yahoo Finance and CompaniesMarketCap.com, this overview demystifies the     │
│  rankings while providing actionable insights. Whether you're a beginner decoding market cap explained or a     │
│  seasoned pro eyeing S&P 500 leaders, let's uncover how these behemoths influenced everything from your         │
│  smartphone to global supply chains. These companies represent balanced opportunities across sectors, blending  │
│  high-growth tech with stable pharma and diversified investments.                                               │
│                                                                                                                 │
│  ## The Power Players: Ranking the Top 10 Biggest Companies in 2023                                             │
│                                                                                                                 │
│  Diving into the top 10 biggest companies in the world in 2023 reveals a landscape tilted heavily toward tech   │
│  innovation, with a few surprises from pharma and diversified conglomerates. These rankings by market cap       │
│  showcase companies that didn't just survive economic 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

TypeError: Markdown expects text, not CrewOutput(raw='# Top 10 Biggest Companies in the World in 2023 by Market Cap\n\n## Introduction\n\nIn 2023, the global economy witnessed unprecedented growth in market capitalization, with the top 10 biggest companies in the world reaching a combined value exceeding $12 trillion—more than the GDP of entire nations like Japan or Germany. This staggering figure underscores the dominance of innovation-driven industries, particularly technology and pharmaceuticals, in shaping our modern world. If you\'re searching for the largest companies by market cap in 2023, you\'ve come to the right place. Market capitalization, or market cap, is calculated by multiplying a company\'s share price by its outstanding shares, offering a snapshot of its overall size and investor confidence far beyond mere revenue figures.\n\nThis list, based on closing prices as of December 31, 2023, highlights not just the sheer scale of these giants but also the trends propelling t

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [13]:
Markdown(result)

TypeError: Markdown expects text, not CrewOutput(raw='# Top 10 Biggest Companies in the World in 2023 by Market Cap\n\n## Introduction\n\nIn 2023, the global economy witnessed unprecedented growth in market capitalization, with the top 10 biggest companies in the world reaching a combined value exceeding $12 trillion—more than the GDP of entire nations like Japan or Germany. This staggering figure underscores the dominance of innovation-driven industries, particularly technology and pharmaceuticals, in shaping our modern world. If you\'re searching for the largest companies by market cap in 2023, you\'ve come to the right place. Market capitalization, or market cap, is calculated by multiplying a company\'s share price by its outstanding shares, offering a snapshot of its overall size and investor confidence far beyond mere revenue figures.\n\nThis list, based on closing prices as of December 31, 2023, highlights not just the sheer scale of these giants but also the trends propelling t

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).